# metadata4Ing

[Metadata4Ing](https://nfdi4ing.pages.rwth-aachen.de/metadata4ing/metadata4ing/) is an ontology developed within the NFDI Consortium [NFDI4Ing](https://nfdi4ing.de/) "with the aim of providing a thorough framework for the semantic description of research data, with a particular focus on engineering sciences and neighbouring disciplines."

The *h5RDMtoolbox* allows translating JSON-LD data into the HDF5 structure and wise versa (see [here](../userguide/wrapper/FAIRAttributes.ipynb)). This way, engineering process metadata can be added easily to the HDF5 file. 

Consider the following example file taken from the *m4i* ontology ["first steps guide"](https://git.rwth-aachen.de/nfdi4ing/metadata4ing/metadata4ing/-/blob/1.2.1/training/first-steps-guide.md), which describes an experimental investigation. You can find the full content of the JSON-LD file [here](min_m4i_ex.jsonld), the following shows an excerpt:

In [ ]:
import h5rdmtoolbox as h5tbx
_ = h5tbx.set_config(auto_create_h5tbx_version=False)

With a single call (`h5tbx.jsonld.to_hdf`) it is possible to read in the JSON-LD data and parse it into the HDF5 structure:

In [ ]:
with h5tbx.File() as h5:
    h5.create_group('metadata')
    h5tbx.jsonld.to_hdf(h5.metadata, source='min_m4i_ex.jsonld')

In [ ]:
print(h5tbx.serialize(h5.hdf_filename, format="ttl", structural=False))

In [ ]:
print(
    h5tbx.serialize(
        h5.hdf_filename,
        format="ttl",
        structural=False,
        context={"@import": "https://w3id.org/nfdi4ing/metadata4ing/m4i_context.jsonld"})
)